In [1]:
import pandas as pd
import numpy as np
import ast

pd.set_option("display.max_columns", None)

In [2]:
data = pd.read_csv('../data/steam_games.csv')
print(f'{data.shape[0]} rows by {data.shape[1]} columns')
data.head()

63723 rows by 21 columns


,Unnamed: 0,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
0,0,10.0,Counter-Strike,Valve,Valve,NaN,184296.0,4785.0,0.0,"10,000,000 .. 20,000,000",11666.0,1107.0,244.0,1068.0,999.0,999.0,0.0,14777.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5371, 'FPS': 4795, 'Multiplayer': 3..."
1,1,20.0,Team Fortress Classic,Valve,Valve,NaN,5232.0,873.0,0.0,"2,000,000 .. 5,000,000",91.0,0.0,18.0,0.0,499.0,499.0,0.0,110.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,2,30.0,Day of Defeat,Valve,Valve,NaN,4881.0,542.0,0.0,"5,000,000 .. 10,000,000",403.0,0.0,26.0,0.0,499.0,499.0,0.0,117.0,"English, French, German, Italian, Spanish - Spain",Action,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,3,40.0,Deathmatch Classic,Valve,Valve,NaN,1791.0,402.0,0.0,"5,000,000 .. 10,000,000",33.0,0.0,6.0,0.0,499.0,499.0,0.0,3.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,4,50.0,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,12486.0,633.0,0.0,"5,000,000 .. 10,000,000",322.0,0.0,127.0,0.0,499.0,499.0,0.0,89.0,"English, French, German, Korean",Action,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."


In [3]:
data.isna().sum()

Unnamed: 0             0
appid                  0
name                  52
developer           6779
publisher           6754
score_rank         63679
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price               6483
initialprice        6483
discount            6483
ccu                    0
languages           6650
genre               6829
tags                   0
dtype: int64

In [4]:
# large amount of nulls in score_rank
data['score_rank'].value_counts()

99.0     16
100.0    12
98.0     12
97.0      4
Name: score_rank, dtype: int64

In [5]:
# price is initialprice - discount. data was collected during a sale so we will be looking at base price in the interest of data accuracy in the long term. we'll move initial price into price and drop initial price and discount
data['price'] = data['initialprice']/100

In [6]:
# we see a lot of imbalanced values here as well that won't help up determine similarity very well
data['userscore'].value_counts()

0.0      63679
100.0        5
95.0         2
80.0         2
46.0         2
51.0         2
94.0         2
68.0         2
77.0         2
84.0         2
71.0         1
59.0         1
82.0         1
66.0         1
83.0         1
88.0         1
57.0         1
92.0         1
60.0         1
69.0         1
70.0         1
87.0         1
96.0         1
78.0         1
55.0         1
73.0         1
97.0         1
63.0         1
61.0         1
53.0         1
76.0         1
98.0         1
65.0         1
Name: userscore, dtype: int64

In [7]:
# for better visualization of a games overall performance we'll make new score columns for positive vs negative ratings and also a total rating column
data['pos_rating_pct'] = (data['positive'] / (data['positive'] + data['negative'])).fillna(0)
data['total_ratings'] = (data['positive'] + data['negative']).astype('int32')

In [8]:
# trimming memory usage
data['appid'] = data['appid'].astype('int32')
data['average_forever'] = data['average_forever'].astype('int32')
data['median_forever'] = data['median_forever'].astype('int32')

In [9]:
# personal preference formatting
data['owners'] = [row.replace(' .. ','-') for row in data['owners']]

average_2weeks and median_2weeks are ever changing values that will be entirely different depending on the time that the data is collected. in a continuously updated app these might be more useful but for now lets not use them. Similarly ccu is the previous days peak player counts and will not be accurate in a long term sense.

In [10]:
data = data[['appid','name','genre','developer','publisher','owners','average_forever','median_forever','pos_rating_pct','total_ratings','price','languages','tags']]

print(data.shape)
data.head()

(63723, 13)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,languages,tags
0,10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,9.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 5371, 'FPS': 4795, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,4.99,"English, French, German, Italian, Spanish - Spain","{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,4.99,"English, French, German, Korean","{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."


In [11]:
data.isna().sum()

appid                 0
name                 52
genre              6829
developer          6779
publisher          6754
owners                0
average_forever       0
median_forever        0
pos_rating_pct        0
total_ratings         0
price              6483
languages          6650
tags                  0
dtype: int64

In [12]:
# since we're going to run into dimmensionality issues later we'll go ahead and drop nulls now. On more powerful hardware we would likely only remove games missing names
data.dropna(inplace=True)

In [13]:
def get_languages(lang_list_series):
    series_as_lists = [col_list.replace('#lang_','').replace('&amp;','').replace('br/','').replace('br /','').replace('lt;','').replace('gt;','').replace('strong*/strong','').split(',') for col_list in lang_list_series]
    languages = []
    for lang_list in series_as_lists:
        for lang_ind in range(len(lang_list)):
            lang_list[lang_ind] = lang_list[lang_ind].strip()
        languages = list(set(languages + lang_list))
    languages.sort()
    languages_condensed = []
    for lang in languages:
        valid = True
        for filt in ['audio', 'not', '*', '-', ';']:
            if filt in lang:
                valid = False
                break
        if valid:
            languages_condensed.append(lang)
    return languages_condensed

In [14]:
def get_genres(genre_list_series):
    series_as_lists = [genre_list.split(',') for genre_list in genre_list_series]
    genres = []
    for genre_list in series_as_lists:
        for genre_ind in range(len(genre_list)):
            genre_list[genre_ind] = genre_list[genre_ind].strip()
        genres = list(set(genres + genre_list))
    genres.sort()
    return genres

In [15]:
def generate_dummy_df(series, columns):
    df = pd.DataFrame()
    for col in columns:
        df[series.name + '_' + col] = [1 if col in row else 0 for row in series]
    return df

In [16]:
lang_df = generate_dummy_df(data['languages'], get_languages(data['languages']))
lang_df['appid'] = list(data['appid'])
lang_df.head()

,languages_Arabic,languages_Bulgarian,languages_Czech,languages_Danish,languages_Dutch,languages_English,languages_Finnish,languages_French,languages_German,languages_Greek,languages_Hungarian,languages_Italian,languages_Japanese,languages_Korean,languages_Norwegian,languages_Not supported,languages_Polish,languages_Portuguese,languages_Romanian,languages_Russian,languages_Simplified Chinese,languages_Slovakian,languages_Spanish,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese,appid
0,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,10
1,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,20
2,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,30
3,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,40
4,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50


In [17]:
data = data.merge(lang_df, on='appid')
print(data.shape)
data.head()

(56766, 42)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,languages,tags,languages_Arabic,languages_Bulgarian,languages_Czech,languages_Danish,languages_Dutch,languages_English,languages_Finnish,languages_French,languages_German,languages_Greek,languages_Hungarian,languages_Italian,languages_Japanese,languages_Korean,languages_Norwegian,languages_Not supported,languages_Polish,languages_Portuguese,languages_Romanian,languages_Russian,languages_Simplified Chinese,languages_Slovakian,languages_Spanish,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese
0,10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,9.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 5371, 'FPS': 4795, 'Multiplayer': 3...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0
1,20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 745, 'FPS': 306, 'Multiplayer': 258...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0
2,30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,4.99,"English, French, German, Italian, Spanish - Spain","{'FPS': 785, 'World War II': 246, 'Multiplayer...",0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 628, 'FPS': 138, 'Classic': 106, 'M...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0
4,50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,4.99,"English, French, German, Korean","{'FPS': 879, 'Action': 321, 'Classic': 250, 'S...",0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
genre_df = generate_dummy_df(data['genre'], get_genres(data['genre']))
genre_df['appid'] = list(data['appid'])
genre_df.head()

,genre_360 V,genre_Accounting,genre_Action,genre_Action Games,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Documentary,genre_Early Access,genre_Education,genre_Episodic,genre_Free to Play,genre_Game Development,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Movie,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Short,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Tutorial,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,appid
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50


In [19]:
data = data.merge(genre_df, on='appid')
print(data.shape)
data.head()

(56778, 76)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,languages,tags,languages_Arabic,languages_Bulgarian,languages_Czech,languages_Danish,languages_Dutch,languages_English,languages_Finnish,languages_French,languages_German,languages_Greek,languages_Hungarian,languages_Italian,languages_Japanese,languages_Korean,languages_Norwegian,languages_Not supported,languages_Polish,languages_Portuguese,languages_Romanian,languages_Russian,languages_Simplified Chinese,languages_Slovakian,languages_Spanish,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese,genre_360 V,genre_Accounting,genre_Action,genre_Action Games,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Documentary,genre_Early Access,genre_Education,genre_Episodic,genre_Free to Play,genre_Game Development,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Movie,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Short,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Tutorial,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing
0,10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,9.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 5371, 'FPS': 4795, 'Multiplayer': 3...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 745, 'FPS': 306, 'Multiplayer': 258...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,4.99,"English, French, German, Italian, Spanish - Spain","{'FPS': 785, 'World War II': 246, 'Multiplayer...",0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,4.99,"English, French, German, Italian, Spanish - Sp...","{'Action': 628, 'FPS': 138, 'Classic': 106, 'M...",0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,4.99,"English, French, German, Korean","{'FPS': 879, 'Action': 321, 'Classic': 250, 'S...",0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
def get_keys(series):
    keys = []
    for item in series:
        if type(item) == dict:
            keys += item.keys()
        keys = list(set(keys))
        keys.sort()
    return keys

In [21]:
def generate_tag_data(series):
    keys = get_keys(series=series)
    df = pd.DataFrame(columns=keys)
    total_rows = len(series)
    for row in series:
        df = df.append(row, ignore_index=True)
        print(f'{df.shape[0]} out of {total_rows} rows completed', end= '\r')
    df_t = df.fillna(0).T
    print('')
    for col in df_t.columns:
        df_t[col] = df_t[col] / df_t[col].max()
        print(f'Scaling game tags for row {col}', end= '\r')
    df = df_t.T
    series = [', '.join(list(row.keys())) for row in series]
    
    return df, series

In [22]:
data['tags'] = [ast.literal_eval(tag_list) if type(ast.literal_eval(tag_list)) == dict else {} for tag_list in data['tags']]
tag_df, data['tags'] = generate_tag_data(data['tags'])
tag_df['appid'] = list(data['appid'])
tag_df.head()

56778 out of 56778 rows completed


,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base-Building,Baseball,Based On A Novel,Basketball,Batman,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Blood,Board Game,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Casual,Cats,Character Action Game,Character Customization,Chess,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Crafting,Creature Collector,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Difficult,Dinosaurs,Diplomacy,Documentary,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dynamic Narration,Dystopian,Early Access,Economy,Education,Electronic,Electronic Music,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Feature Film,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football,Foreign,Free to Play,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Indie,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Jet,Kickstarter,LEGO,LGBTQ+,Lara Croft,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Management,Mars,Martial Arts,Massively Multiplayer,Masterpiece,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Photo Editing,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Pool,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle-Platformer,PvE,PvP,Quick-Time Events,RPG,RPGMaker,RTS,Racing,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Rogue-like,Rogue-lite,Roguelike Deckbuilder,Roguevania,Romance,Rome,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Short,Side Scroller,Silent Protagonist,Simulation,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snow,Snowboarding,Soccer,Social Deduction,Software,Software Training,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fighter,Spelling,Split Screen,Sports,Stealth,Steam Machine,Steampunk,Story Rich,Strategy,Strategy RPG,Stylized,Submarine,Superhero,Supernatural,Surreal,Survival,Survival Horror,Swordplay,Tabletop,Tactical,Tactical RPG,Tanks,Te

In [23]:
data = data.merge(tag_df.fillna(0), on='appid')
print(data.shape)
data.head()

(57018, 506)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,languages,tags,languages_Arabic,languages_Bulgarian,languages_Czech,languages_Danish,languages_Dutch,languages_English,languages_Finnish,languages_French,languages_German,languages_Greek,languages_Hungarian,languages_Italian,languages_Japanese,languages_Korean,languages_Norwegian,languages_Not supported,languages_Polish,languages_Portuguese,languages_Romanian,languages_Russian,languages_Simplified Chinese,languages_Slovakian,languages_Spanish,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese,genre_360 V,genre_Accounting,genre_Action,genre_Action Games,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Documentary,genre_Early Access,genre_Education,genre_Episodic,genre_Free to Play,genre_Game Development,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Movie,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Short,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Tutorial,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base-Building,Baseball,Based On A Novel,Basketball,Batman,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Blood,Board Game,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Casual,Cats,Character Action Game,Character Customization,Chess,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Crafting,Creature Collector,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Difficult,Dinosaurs,Diplomacy,Documentary,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dynamic Narration,Dystopian,Early Access,Economy,Education,Electronic,Electronic Music,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Feature Film,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football,Foreign,Free to Play,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Indie,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Jet,Kickstarter,LEGO,LGBTQ+,Lara Croft,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Management,Mars,Martial Arts,Massively Multiplayer,Masterpiece,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,

In [24]:
data[data['appid'].duplicated()]

,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,languages,tags,languages_Arabic,languages_Bulgarian,languages_Czech,languages_Danish,languages_Dutch,languages_English,languages_Finnish,languages_French,languages_German,languages_Greek,languages_Hungarian,languages_Italian,languages_Japanese,languages_Korean,languages_Norwegian,languages_Not supported,languages_Polish,languages_Portuguese,languages_Romanian,languages_Russian,languages_Simplified Chinese,languages_Slovakian,languages_Spanish,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese,genre_360 V,genre_Accounting,genre_Action,genre_Action Games,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Documentary,genre_Early Access,genre_Education,genre_Episodic,genre_Free to Play,genre_Game Development,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Movie,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Short,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Tutorial,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base-Building,Baseball,Based On A Novel,Basketball,Batman,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Blood,Board Game,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Casual,Cats,Character Action Game,Character Customization,Chess,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Crafting,Creature Collector,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Difficult,Dinosaurs,Diplomacy,Documentary,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dynamic Narration,Dystopian,Early Access,Economy,Education,Electronic,Electronic Music,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Feature Film,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football,Foreign,Free to Play,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Indie,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Jet,Kickstarter,LEGO,LGBTQ+,Lara Croft,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Management,Mars,Martial Arts,Massively Multiplayer,Masterpiece,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,

In [25]:
data.drop(labels=data[data['appid'].duplicated()].index, inplace=True)

In [26]:
# splitting into 3 parts for github friendly filesizes
data.iloc[:len(data)//3,:].to_csv('../data/steam_games_cleaned_1.csv', index = False)
data.iloc[len(data)//3 : len(data)//3*2, : ].to_csv('../data/steam_games_cleaned_2.csv', index = False)
data.iloc[len(data)//3*2 : , : ].to_csv('../data/steam_games_cleaned_3.csv', index = False)